In [ ]:
# Market Volume and Volatility Screener
# This notebook analyzes cryptocurrency markets to identify high-volume and high-volatility trading opportunities
# It uses CLOB (Central Limit Order Book) data to fetch and analyze candle data from various trading pairs

from core.data_sources.clob import CLOBDataSource
import warnings
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from core.features.candles.volatility import Volatility, VolatilityConfig
from core.features.candles.volume import Volume, VolumeConfig

warnings.filterwarnings("ignore")

# Initialize CLOB data source for fetching market data
clob = CLOBDataSource()

In [ ]:
# Configuration Parameters
# These constants control the data fetching and analysis behavior

# Market configuration
CONNECTOR_NAME = "binance_perpetual"  # Exchange connector to use
QUOTE_ASSET = "USDT"                  # Quote currency for trading pairs
INTERVAL = "1m"                       # Candle interval (1 minute)

# Technical analysis parameters
VOLATILITY_WINDOW = 20        # Window size for volatility calculations (20 periods)
VOLUME_SHORT_WINDOW = 20      # Short-term volume moving average window
VOLUME_LONG_WINDOW = 50       # Long-term volume moving average window

# Data fetching configuration
FETCH_CANDLES = False         # Set to True to fetch fresh data from exchange
DAYS = 15                    # Number of days of historical data to fetch
BATCH_CANDLES_REQUEST = 2    # Number of concurrent requests when fetching
SLEEP_REQUEST = 2.0          # Sleep time between batch requests (seconds)

# Data Fetching and Caching
# This section either fetches fresh data from the exchange or loads cached data
# Caching helps avoid unnecessary API calls and speeds up repeated analysis

In [ ]:
# Fetch or load candle data
# If FETCH_CANDLES is True: Downloads fresh data from the exchange
# If FETCH_CANDLES is False: Loads previously cached data

if FETCH_CANDLES:
    # Get all available trading rules from the exchange
    trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
    
    # Filter trading pairs to exclude those with matching quote asset
    # This ensures we only get USDT-quoted pairs (e.g., BTC-USDT, ETH-USDT)
    trading_pairs = [
        trading_pair for trading_pair in trading_rules.get_all_trading_pairs() 
        if trading_pair.split("-")[1] != QUOTE_ASSET
    ]
    
    # Fetch candle data for all trading pairs
    # Uses batch processing to avoid rate limiting
    candles = await clob.get_candles_batch_last_days(
        CONNECTOR_NAME, 
        trading_pairs, 
        INTERVAL, 
        DAYS, 
        BATCH_CANDLES_REQUEST, 
        SLEEP_REQUEST
    )
    
    # Save fetched data to cache for future use
    clob.dump_candles_cache()
    print(f"Fetched data for {len(candles)} trading pairs")
else:
    # Load previously cached data
    clob.load_candles_cache()
    print("Loaded data from cache")

In [ ]:
# Filter candles to only include the desired connector and interval
# The cache may contain data from multiple connectors and intervals
candles = [
    value for key, value in clob.candles_cache.items() 
    if key[2] == INTERVAL and key[0] == CONNECTOR_NAME
]
print(f"Found {len(candles)} candle datasets for {CONNECTOR_NAME} with {INTERVAL} interval")

In [ ]:
# Market Analysis: Calculate Volume and Volatility Metrics
# This replaces the utils.generate_report function with inline code for better transparency

# Initialize configurations for technical indicators
volatility_config = VolatilityConfig(window=VOLATILITY_WINDOW)
volume_config = VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW)

# Process each trading pair and calculate metrics
report = []
for candle in candles:
    try:
        # Add volatility and volume features to the candle data
        candle.add_features([
            Volatility(volatility_config), 
            Volume(volume_config)
        ])
        df = candle.data
        
        # Calculate volatility metrics
        # - volatility: Standard deviation of returns
        # - natr: Normalized Average True Range (volatility relative to price)
        # - bb_width: Bollinger Bands width (another volatility measure)
        mean_volatility = df['volatility'].mean()
        mean_natr = df['natr'].mean()
        mean_bb_width = df['bb_width'].mean()
        
        # Get the latest trend signal
        # rolling_buy_sell_imbalance shows buying vs selling pressure
        latest_trend = df['rolling_buy_sell_imbalance_short'].iloc[-1]
        
        # Calculate volume metrics
        # Average hourly volume gives us liquidity information
        total_volume_usd = df['volume_usd'].sum()
        total_hours = (df.index[-1] - df.index[0]).total_seconds() / 3600
        average_volume_per_hour = total_volume_usd / total_hours
        
        # Price range analysis
        # These metrics help identify where price is within its recent range
        max_price = df['close'].max()
        min_price = df['close'].min()
        range_price = max_price - min_price
        
        # Calculate how far current price is from the high (as percentage)
        range_price_pct = (max_price - df['close'].iloc[-1]) / df['close'].iloc[-1]
        
        # Current position in the range (0 = at min, 1 = at max)
        current_position = (max_price - df['close'].iloc[-1]) / range_price if range_price > 0 else 0
        
        # Composite score combining volatility, volume, and price position
        # Higher scores indicate more attractive trading opportunities
        score = mean_natr * average_volume_per_hour * current_position * range_price_pct
        
        report.append({
            'trading_pair': candle.trading_pair,
            'mean_volatility': mean_volatility,
            'mean_natr': mean_natr,
            'mean_bb_width': mean_bb_width,
            'latest_trend': latest_trend,
            'average_volume_per_hour': average_volume_per_hour,
            'current_position': current_position,
            'range_price_pct': range_price_pct,
            'score': score
        })
    except Exception as e:
        print(f"Error processing {candle.trading_pair}: {e}")
        continue

# Convert to DataFrame and normalize scores
report_df = pd.DataFrame(report)
max_score = report_df['score'].max()
report_df['normalized_score'] = report_df['score'] / max_score if max_score > 0 else 0
report_df.drop(columns=['score'], inplace=True)

# Display the report sorted by normalized score
print(f"\\nAnalyzed {len(report_df)} trading pairs")
report_df.sort_values('normalized_score', ascending=False).head(10)

In [ ]:
## Market Filtering Strategy
# Filter markets based on quantile thresholds to identify the best trading opportunities
# Markets must meet minimum volume and volatility requirements and have favorable price positioning

# Filtering parameters (adjust these to be more or less selective)
VOLUME_QUANTILE = 0.5          # Markets must be above 50th percentile in volume
NATR_QUANTILE = 0.5            # Markets must be above 50th percentile in volatility
CURRENT_POSITION_THRESHOLD = 0.8  # Price must be in upper 80% of range (closer to highs)
TOP_X_MARKETS = 20             # Number of top markets to analyze

# Apply filters to identify high-opportunity markets
# These are markets with good liquidity, high volatility, and favorable price positioning
top_markets = report_df[
    (report_df['average_volume_per_hour'] > report_df['average_volume_per_hour'].quantile(VOLUME_QUANTILE)) &
    (report_df['mean_natr'] > report_df['mean_natr'].quantile(NATR_QUANTILE)) &
    (report_df['current_position'] > CURRENT_POSITION_THRESHOLD)
]

# Sort by current position (markets closest to their highs) and take top X
top_markets = top_markets.sort_values(by='current_position', ascending=False).head(TOP_X_MARKETS)

# Create a dictionary of candles for selected markets for detailed analysis
top_markets_candles = {
    candle.trading_pair: candle 
    for candle in candles 
    if candle.trading_pair in top_markets["trading_pair"].values
}

print(f"\\n🎯 FILTERED MARKETS")
print("=" * 50)
print(f"Applied filters:")
print(f"  - Volume > {VOLUME_QUANTILE*100}th percentile")
print(f"  - Volatility (NATR) > {NATR_QUANTILE*100}th percentile")
print(f"  - Price position > {CURRENT_POSITION_THRESHOLD} (near recent highs)")
print(f"\\nMarkets meeting criteria: {len(top_markets_candles)}")
print(f"Selected markets: {list(top_markets_candles.keys())}")

In [ ]:
## Individual Market Deep Dive
# Display detailed chart for a selected market from the filtered list

if len(top_markets_candles) > 0:
    # Select the first market from our filtered list for detailed view
    selected_market = list(top_markets_candles.keys())[0]
    print(f"Displaying detailed chart for: {selected_market}")
    print(f"Market rank: #1 out of {len(top_markets_candles)} filtered markets")
    
    # Display the interactive chart with all indicators
    top_markets_candles[selected_market].plot()
    
    # Print key statistics for this market
    market_stats = report_df[report_df['trading_pair'] == selected_market].iloc[0]
    print(f"\\n📊 {selected_market} KEY METRICS:")
    print(f"  - Normalized Score: {market_stats['normalized_score']:.4f}")
    print(f"  - Average Hourly Volume: ${market_stats['average_volume_per_hour']:,.0f}")
    print(f"  - Mean NATR: {market_stats['mean_natr']:.6f}")
    print(f"  - Current Position in Range: {market_stats['current_position']:.2%}")
    print(f"  - Latest Trend Signal: {market_stats['latest_trend']:.4f}")
else:
    print("No markets available for display. Adjust filtering criteria in the Market Filtering section.")

In [ ]:
# Correlation Analysis between Volume and Volatility metrics
# This helps understand if high volume coincides with high volatility

# Calculate correlations
correlation_data = report_df[['average_volume_per_hour', 'mean_volatility', 'mean_natr', 'mean_bb_width', 'latest_trend', 'current_position']]

# Log transform volume for better correlation analysis (due to wide range)
correlation_data['log_volume'] = np.log10(correlation_data['average_volume_per_hour'] + 1)

# Calculate correlation matrix
corr_matrix = correlation_data.corr()

# Create correlation heatmap
fig_corr = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=np.round(corr_matrix.values, 2),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Correlation")
))

fig_corr.update_layout(
    title="Correlation Matrix: Volume and Volatility Metrics",
    height=600,
    xaxis_title="",
    yaxis_title=""
)

fig_corr.show()

# Key correlation insights
print("\\n🔍 CORRELATION INSIGHTS")
print("=" * 50)

# Volume-Volatility correlation
vol_natr_corr = correlation_data[['log_volume', 'mean_natr']].corr().iloc[0, 1]
print(f"\\nVolume-NATR Correlation: {vol_natr_corr:.3f}")
if abs(vol_natr_corr) > 0.5:
    print("  → Strong correlation: High volume markets tend to have", 
          "high volatility" if vol_natr_corr > 0 else "low volatility")
elif abs(vol_natr_corr) > 0.3:
    print("  → Moderate correlation detected")
else:
    print("  → Weak correlation: Volume and volatility are relatively independent")

# Volatility metrics correlation
vol_metrics_corr = correlation_data[['mean_volatility', 'mean_natr', 'mean_bb_width']].corr()
print(f"\\nVolatility Metrics Consistency:")
print(f"  - Volatility vs NATR: {vol_metrics_corr.iloc[0, 1]:.3f}")
print(f"  - Volatility vs BB Width: {vol_metrics_corr.iloc[0, 2]:.3f}")
print(f"  - NATR vs BB Width: {vol_metrics_corr.iloc[1, 2]:.3f}")

# Trend and position correlation
trend_pos_corr = correlation_data[['latest_trend', 'current_position']].corr().iloc[0, 1]
print(f"\\nTrend-Position Correlation: {trend_pos_corr:.3f}")
if trend_pos_corr > 0:
    print("  → Positive correlation: Bullish trends associated with higher price positions")
else:
    print("  → Negative correlation: Bearish trends associated with higher price positions")

# Statistical significance of volume-volatility relationship
from scipy import stats
# Perform regression analysis
slope, intercept, r_value, p_value, std_err = stats.linregress(
    correlation_data['log_volume'], 
    correlation_data['mean_natr']
)
print(f"\\n📊 REGRESSION ANALYSIS (Log Volume vs NATR)")
print(f"  - R-squared: {r_value**2:.3f}")
print(f"  - P-value: {p_value:.4f}")
if p_value < 0.05:
    print(f"  - Relationship is statistically significant!")
else:
    print(f"  - Relationship is not statistically significant")

## Correlation Analysis: Volume-Volatility Relationship
Understanding the relationship between volume and volatility helps in strategy development

## Time Series Analysis: Spotting Volatility Conditions
Analyze how volatility evolves over time to identify specific market conditions and potential trading opportunities

## Visualizations: Volume and Volatility Distributions
Analyzing the distribution of volume and volatility across all markets helps identify outliers and understand market structure

In [15]:
VOLUME_QUANTILE = 0.5
NATR_QUANTILE = 0.5
CURRENT_POSITION_THRESHOLD = 0.8
TOP_X_MARKETS = 20

top_markets = report[(report['average_volume_per_hour'] > report['average_volume_per_hour'].quantile(VOLUME_QUANTILE)) &
                     (report['mean_natr'] > report['mean_natr'].quantile(NATR_QUANTILE)) &
                     (report['current_position'] > CURRENT_POSITION_THRESHOLD)]
top_markets = top_markets.sort_values(by='current_position', ascending=False).head(TOP_X_MARKETS)
top_markets_candles = {candle.trading_pair: candle for candle in candles if candle.trading_pair in top_markets["trading_pair"].values}
print(top_markets_candles.keys())


dict_keys(['ARB-USDT', 'ADA-USDT', 'LINK-USDT'])


In [18]:
list(top_markets_candles.values())[0].plot()